In [2]:
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import statsmodels.formula.api as sm
import statsmodels.api as smf
from scipy.stats import gamma,norm,rayleigh,describe,scoreatpercentile

%matplotlib inline

In [3]:
from utils.Run_LCA_model import Run_LCA_model
from utils.Parameter_values_dict import Parameter_names

In [4]:
buildings_path = '..\shapefiles\\building_block_people_WGS_3_int.csv'
logistics = 'optimal'
analysis = 'Sensitivity'
acid_type = 'Sulfuric'
n_collection = 1
n_regen = 20
for direction in ['plus', 'minus']:
    Result_Energy_m3 = pd.DataFrame()
    Result_GHG_m3 = pd.DataFrame()
    Result_COST_m3 = pd.DataFrame()
    for param in Parameter_names:
        Total_Energy_m3, Total_GHG_m3, Total_COST_m3 = Run_LCA_model(buildings_path, n_regen, n_collection, logistics, analysis,acid_type, parameter = param, direction = direction)
        Total_Energy_m3['param'], Total_GHG_m3['param'], Total_COST_m3['param'] = param, param, param
        Result_Energy_m3 = Result_Energy_m3.append(Total_Energy_m3)
        Result_GHG_m3 = Result_GHG_m3.append(Total_GHG_m3)
        Result_COST_m3 = Result_COST_m3.append(Total_COST_m3)

    Result_Energy_m3["sum"] = (Result_Energy_m3.sum(axis=1)) - Result_Energy_m3["n_facilities"]
    Result_GHG_m3["sum"] = (Result_GHG_m3.sum(axis=1)) - Result_GHG_m3["n_facilities"]
    Result_COST_m3["sum"] = (Result_COST_m3.sum(axis=1)) - Result_COST_m3["n_facilities"]

    if direction == 'plus':
        Result_Energy_param_plus = Result_Energy_m3[['sum', 'param']]
        Result_GHG_param_plus = Result_GHG_m3[['sum', 'param']]
        Result_COST_param_plus = Result_COST_m3[['sum', 'param']]
    if direction == 'minus':
        Result_Energy_param_minus = Result_Energy_m3[['sum', 'param']]
        Result_GHG_param_minus = Result_GHG_m3[['sum', 'param']]
        Result_COST_param_minus = Result_COST_m3[['sum', 'param']]

In [5]:
Sensitivity_Energy = pd.merge(Result_Energy_param_plus, Result_Energy_param_minus, on = 'param')
Sensitivity_GHG = pd.merge(Result_GHG_param_plus, Result_GHG_param_minus, on = 'param')
Sensitivity_COST = pd.merge(Result_COST_param_plus, Result_COST_param_minus, on = 'param')

In [6]:
Sensitivity_Energy['Sensitivity'] = abs(Sensitivity_Energy['sum_y'] - Sensitivity_Energy['sum_x'])*5
Sensitivity_GHG['Sensitivity'] = abs(Sensitivity_GHG['sum_y'] - Sensitivity_GHG['sum_x'])*5
Sensitivity_COST['Sensitivity'] = abs(Sensitivity_COST['sum_y'] - Sensitivity_COST['sum_x'])*5

In [7]:
Sensitivity_Energy.sort(columns='Sensitivity', ascending=False)[:8]

C:\Users\Olga\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,sum_x,param,sum_y,Sensitivity
0,11.071583,percent_served,9.987011,5.422860
13,15.542857,urine_production,16.454131,4.556366
56,16.259919,sulphuric_acid_energy,15.634390,3.127643
10,15.661398,adsorption_density,16.247259,2.929303
23,16.146343,transport_energy_MJ_km,15.747967,1.991877
9,15.812410,resin_lifetime,16.093128,1.403588
5,16.086870,resin_energy_MJ_kg,15.807440,1.397151
11,16.056664,time_between_catridge_regeneration,15.837646,1.095090


In [8]:
Sensitivity_GHG.sort(columns='Sensitivity', ascending=False)[:8]

C:\Users\Olga\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,sum_x,param,sum_y,Sensitivity
0,1.726679,percent_served,1.449295,1.386922
57,3.053288,sulphuric_acid_GHG,2.894330,0.794789
10,2.911775,adsorption_density,3.038958,0.635919
24,3.024965,transport_GHG_kg_km,2.922654,0.511556
13,2.935473,urine_production,3.021882,0.432043
14,3.012012,household_size,2.932423,0.397944
9,2.957567,resin_lifetime,2.991405,0.169186
6,2.990575,resin_GHG_kg_kg,2.957043,0.167658


In [9]:
Sensitivity_COST.sort(columns='Sensitivity', ascending=False)[:8]

C:\Users\Olga\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,sum_x,param,sum_y,Sensitivity
13,39.990861,urine_production,46.390793,31.999664
0,17.733789,percent_served,12.156523,27.886328
75,43.437877,facility_cost_regression,42.222658,6.076095
10,42.476698,adsorption_density,43.201589,3.624456
11,43.124087,time_between_catridge_regeneration,42.536448,2.938193
61,43.009095,sulphuric_acid_cost,42.651440,1.788276
9,42.797977,resin_lifetime,42.865249,0.336357
4,42.863799,resin_cost_kg,42.796736,0.335316
